In [1]:
# INPUTS
# fileloc = location of point cloud
# saveloc = folder for outputs
# mask = denotes whether file is a mask image. mask = 1 skips rasterisation step and resizes to match image dimensions.
# res = multiplier on resolution of rasterised point cloud imagedimension = defaultdimension*res
# dim = if given, resulting image is cropped into dimxdim images and put into relevant mask or image folder ready for network training. 

# INPUT FILES
# if mask = 0, pointcloud or .tiff format image at location: fileloc
# if mask = 1, mask at location: saveloc+"mask.tiff" 
# if mask = 1, image at location: saveloc+ "raster.tiff"

# OUTPUTS
# noim = number of images created

# OUTPUT FILES
# if mask = 0 and input is point cloud, rastered image at location: saveloc+ "raster.tiff"
# if mask = 0 and input is an image, rastered image at location: saveloc+ "raster.tiff"
# if mask = 1, resized mask at location: saveloc+"mask.tiff"
# if dim exists and mask = 0, new folder (if not exists): 'croppedimages/'
# if dim exists and mask = 1, new folder (if not exists): 'croppedmasks/'
# if dim exists and mask = 0, numbered image crops of size dimxdim at location: saveloc+croppedimages/+ str(i) +".tiff"
# if dim exists and mask = 1, numbered mask crops of size dimxdim at location: saveloc+croppedmasks/+ str(i) +".tiff"


#OUTPUT FILES

#def raster(fileloc,saveloc,mask = 0,res = 1, dim = None):


from PIL import Image, ImageOps

import os
import cv2
import laspy
import numpy as np
import torch
import matplotlib.pyplot as plt
os.chdir("../data/imtest/")



In [2]:
fileloc = "testflatten.las"
las = laspy.read(fileloc)
print(np.unique(las.classification)) 


[0]


In [3]:
for dimension in las.point_format.dimensions:
    print(dimension.name)

X
Y
Z
intensity
return_number
number_of_returns
scan_direction_flag
edge_of_flight_line
classification
synthetic
key_point
withheld
scan_angle_rank
user_data
point_source_id
gps_time
red
green
blue


In [4]:
las.header.scale

array([0.001, 0.001, 0.001])

In [5]:
las.header.offset

array([-4., -1.,  0.])

In [6]:
points = np.vstack((las.x, las.y, las.z)).transpose()  

In [7]:
points

array([[-3.25 , -0.048,  5.506],
       [-3.35 , -0.041,  5.554],
       [-3.321, -0.017,  5.884],
       ...,
       [ 2.98 , -0.006,  1.706],
       [ 3.061, -0.022,  1.699],
       [ 2.873, -0.471,  1.594]])

In [8]:

print(np.unique(las.classification)) 
for dimension in las.point_format.dimensions:
    print(dimension.name)
points = np.vstack((las.X, las.Y, las.Z)).transpose()    
nop = len(points[:,0])
print('number of points = {}'.format(nop))
xmin = np.argmin(points[:,0])
xminv= points[xmin,0]
xmax = np.argmax(points[:,0])
xmaxv= points[xmax,0]
ymin = np.argmin(points[:,1])
yminv= points[ymin,1]
ymax = np.argmax(points[:,1])
ymaxv= points[ymax,1]
ywidth =ymaxv-yminv
xwidth =xmaxv-xminv
ratio = xwidth/ywidth
print(ywidth,xwidth,ratio)
nx = (ratio*nop)**(1/2)
nx = nx*res

nx = int(nx)
xbinsize = (xmaxv-xminv)/nx
ybinsize = xbinsize
ny = (ymaxv-yminv)/ybinsize
ny = int(ny)+1
nx = nx+1

image = np.zeros((nx,ny))
npix = image.shape[0]*image.shape[1]
print('number of pixels = {}'.format(npix))
print('width in pixels = {}'.format(nx))
print('height in pixels = {}'.format(ny))
xp = np.floor((points[:,0]-xminv)/xbinsize).astype(int)
yp = np.floor((points[:,1]-yminv)/ybinsize).astype(int)
zp = points[:,2]
image = np.zeros((nx,ny))
imagenum = np.zeros((nx,ny))
for i in range(len(xp)):
    imagenum[xp[i],yp[i]]+=1
a = 0
for i in range(len(xp)):
    a = imagenum[xp[i],yp[i]]
    #   if a != 0:
    image[xp[i],yp[i]]+=zp[i]/a
image = image - image.min()
imagenumb = imagenum.astype(bool)
imagenumb = 0 -imagenumb
plt.imshow(imagenumb)
plt.imshow(image)
# outputnum2 = Image.fromarray(imagenumb)
# outputnum2 = outputnum2.convert("L")
#imagenumb2 = np.array(outputnum2)
#output = Image.fromarray(image)
#image2 = np.array(output)
#infilled = cv2.inpaint(image2, imagenum.astype('uint8'), 3, cv2.INPAINT_NS)
for a in range(len(imagenum[0,:])):
    for b in range(len(imagenum[:,0])):
        if imagenum[b,a] == 0:
            c = 0
            d = 0
            E = [-1, 0, 1]
            F = [-1, 0, 1]
            for e in E:
                for f in F:
                    try:
                        if imagenum[b+e,a+f] != 0:
                            d+=image[b+e,a+f]
                            c+=1
                    except:
                        pass
            if d !=0:

                image[b,a]=d/c

#   for a in range(len(imagenum[0,:])-1,0,-1):
#       for b in range(len(imagenum[:,0])-1,0,-1):
#           if imagenum[b,a] == 0:#
#tese
#                try:
#                    image[b,a]=image[b+1,a]
#                except:
#                    b   


outputf = Image.fromarray(image)
print('mean pixel value = {}'.format(image.mean()))
outputf.save(saveloc+"raster.tiff")
#np.save(saveloc+"raster.tiff",image)

            


[0]
X
Y
Z
intensity
return_number
number_of_returns
scan_direction_flag
edge_of_flight_line
classification
synthetic
key_point
withheld
scan_angle_rank
user_data
point_source_id
gps_time
red
green
blue
number of points = 1112805
597 6536 10.948073701842546


NameError: name 'res' is not defined

In [25]:
print(ywidth,xwidth,ratio)

0 6000 inf


In [ ]:
elif mask == 1:
    image = Image.open(saveloc+"raster.tiff")
    nx,ny = image.size
    outputf = Image.open(fileloc)
    outputf = outputf.resize((nx,ny))
    outputf.save(saveloc+"mask.tiff")
elif mask == 0 and "tiff" in fileloc:
    outputf = Image.open(fileloc)
    nx,ny = outputf.size
    outputf = outputf.resize((int(nx*res),int(ny*res)))
    outputf.save(saveloc+"raster.tiff")        
noim = 0
if dim != None:
        if mask == 0:
            newpath = r'croppedimages/'
        else:
            newpath = r'croppedmasks/'
        if not os.path.exists(saveloc+newpath):
            os.makedirs(saveloc+newpath)
        # pic = outputf
        if "tiff" in fileloc:
                nx,ny = outputf.size
                outputf = outputf.resize((int(nx*res),int(ny*res)))
        #pic = ImageOps.grayscale(pic)
        pic = outputf
        w = pic.size[0]
        nx = int(w/dim)
        h = pic.size[1]
        ny = int(h/dim)
        i = 0
        print("cropping masks")
        for x in range(0, nx):
            for y in range(0, ny):
                i = i+1
                box = (x*dim, y*dim, dim*(1+x) , dim*(1+y))
                region = pic.crop(box)
                #region.save("test2/croppedimagestargetfullres2/"+ str(i) +".png")
                region.save(saveloc+newpath+ str(i) +".tiff")
                noim +=1